# Данные

В исследовании используются данные со статистикой конверсии пользователей сайта.

Краткое описание:

"Компания разработала новую посадочную страницу с целью увеличения продаж своего продукта. Было решено провести А/Б-тестирование, чтобы узнать, какая из версий посадочной страницы покажет себя лучше. Пользователей собрали в две группы одинакового размера. Экспериментальной группе (treatment group) была показана новая посадочная страница, контрольной группе (control group) - старая."

Данные можно найти по ссылке: https://www.kaggle.com/datasets/putdejudomthai/ecommerce-ab-testing-2022-dataset1.

# Какие данные представлены?

- user_id: уинкальный идентификатор пользователя
- timestamp: время посещения страницы
- group: группа (экспериментальная или контрольная)
- landing_page: посадочная страница (старая - old_page либо новая - new_page)
- converted: статус пользователя после просмотра страницы (1 - совершил покупку, 0 - нет)

# Задача

Выяснить, оказывает ли новый дизайн посадочной страницы какое-либо влияние на конверсию пользователей (есть ли статистически значимая разница между конверсиями)?

# Предобработка данных

In [1]:
# Загрузка библиотек
import pandas as pd
import numpy as np

In [2]:
# Загрузка данных
df = pd.read_csv('data/ab_data.csv')
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1


Сколько записей в данных? Какого типа переменные?

In [3]:
print(f'Размер датасета: {df.shape[0]} строк и {df.shape[1]} столбцов.')
df.info()

Размер датасета: 294480 строк и 5 столбцов.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294480 entries, 0 to 294479
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294480 non-null  int64 
 1   timestamp     294480 non-null  object
 2   group         294480 non-null  object
 3   landing_page  294480 non-null  object
 4   converted     294480 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


Есть ли в данных пропуски?

In [4]:
df.isnull().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

Посмотрим, сколько уникальных значений содержится в каждой колонке. Выведем на экран по 5 уникальных значений из каждой колонки.

In [5]:
for col in df.columns: 
  unique_vals = df[col].unique() # Найдем уникальные значения
  len_unique_vals = len(unique_vals) # Посчитаем количество уникальных значений

  # Выведем результаты
  print(col)
  print(f'Количество уникальных значений = {len_unique_vals}: {unique_vals[:5]}')
  print('#######################################', '\n')

user_id
Количество уникальных значений = 290585: [851104 804228 661590 853541 864975]
####################################### 

timestamp
Количество уникальных значений = 35993: ['11:48.6' '01:45.2' '55:06.2' '28:03.1' '52:26.2']
####################################### 

group
Количество уникальных значений = 2: ['control' 'treatment']
####################################### 

landing_page
Количество уникальных значений = 2: ['old_page' 'new_page']
####################################### 

converted
Количество уникальных значений = 2: [0 1]
####################################### 



Есть ли в данных повторяющиеся строки?

In [6]:
print(f'Число повторов в данных: {df.duplicated().sum()}')

Число повторов в данных: 0


Есть ли повторяющиеся идентификаторы пользователей?

In [7]:
df[df['user_id'].duplicated()]

,user_id,timestamp,group,landing_page,converted
2656,698120,13:42.6,control,old_page,0
2893,773192,55:59.6,treatment,new_page,0
7500,899953,06:54.1,control,new_page,0
8036,790934,32:20.3,treatment,new_page,0
10218,633793,16:00.7,treatment,old_page,0
...,...,...,...,...,...
294309,787083,15:21.0,control,old_page,0
294328,641570,59:27.7,control,old_page,0
294331,689637,34:28.3,control,new_page,0
294355,744456,32:07.1,treatment,new_page,0


Есть ли в данных записи с несоответствиями в колонках group и landing_page?

Группе treatment должна была показываться новая посадочная страница, группе control - старая.

In [8]:
df_mismatch_in_treatment = df[(df['group']=='treatment') & (df['landing_page']=='old_page')]

df_mismatch_in_control = df[(df['group']=='control') & (df['landing_page']=='new_page')]

n_mismatch = df_mismatch_in_treatment.shape[0] + df_mismatch_in_control.shape[0]

print(f'Сколько раз экспериментальной группе была показана старая посадочная страница: {df_mismatch_in_treatment.shape[0]}')
print(f'Сколько раз контрольной группе была показана новая посадочная страница: {df_mismatch_in_control.shape[0]}')
print(f'Общее число ошибок: {n_mismatch}')

Сколько раз экспериментальной группе была показана старая посадочная страница: 1965
Сколько раз контрольной группе была показана новая посадочная страница: 1928
Общее число ошибок: 3893


Было найдено 3895 повторяющихся идентификаторов пользователей и 3893 несоответствия между группой и посадочной страницей.

По каким-то причинам некоторым пользователям была показана неправильная посадочная страница. Исключим таких пользователей из исследования.

In [9]:
df_cleaned = df[(df["group"]=="treatment")&(df["landing_page"]=="new_page") | (df["group"]=="control")&(df["landing_page"]=="old_page")]
df_cleaned.shape

(290587, 5)

В данных остается еще два дубликата. Данные дубликаты появились из-за повторного посещения пользователями посадочной страницы.

In [10]:
df_cleaned[df_cleaned['user_id'].duplicated()]

,user_id,timestamp,group,landing_page,converted
2893,773192,55:59.6,treatment,new_page,0
294478,759899,20:29.0,treatment,new_page,0


In [11]:
df_cleaned[df_cleaned['user_id'] == 773192]

,user_id,timestamp,group,landing_page,converted
1899,773192,37:58.8,treatment,new_page,0
2893,773192,55:59.6,treatment,new_page,0


In [12]:
df_cleaned[df_cleaned['user_id'] == 759899]

,user_id,timestamp,group,landing_page,converted
250001,759899,07:36.1,treatment,new_page,0
294478,759899,20:29.0,treatment,new_page,0


In [13]:
# Сохранение датафрейма
df_cleaned.to_csv('data/df_cleaned.csv', index=False)